#### 네이버 카페 크롤러
가입이 필수인 카페들은 조건을 만족시켜줘야 함
1. 크롬 드라이버 제어(크롬창 켜기)
2. 네이버 로그인(아이디 입력 -< 비밀번호 입력 -> 로그인 버튼 클릭 순서로 작업 진행)

In [1]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install bs4
!pip install webdriver_manager
!pip install selenium==3.141.0

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install webdriver_manager

Defaulting to user installation because normal site-packages is not writeable


In [5]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import csv
import pandas as pd
import random
from bs4 import BeautifulSoup as bs

In [6]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [7]:
#크롬 드라이버 제어
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dex-shm-usage')

# 현재 컴퓨터 크롬 드라이버 위치
chrome_path = 'chromedriver.exe'    # 본인의 크롬 드라이버 위치 입력

# 자동화된 크롬 창 실행
driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)


C:\Users\user\AppData\Local\Temp\ipykernel_16584\161475440.py:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)


In [8]:
# 네이버 로그인 페이지 접속
login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
driver.get(login_url)
driver.implicitly_wait(10)

# 본인의 아이디, 비밀번호를 각 변수에 저장
my_id = 'dlwnsrl8964'
my_pw = 'dustnrh31322!'

# 보통 사이트에서는 send_key 사용하는데 네이버는 자동입력방지 기능 때문에 다른 방법 사용
# 아이디, 비밀번호 입력 (네이버에 로그인 할 경우 'send_keys()' 함수가 아니라 'execute_script()' 사용)
#네이버에서 send_key()를 쓰면 로그인에서 자동입력방지기능이 나타남.
driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
time.sleep(1)

# 로그인 버튼 클릭
driver.find_element('id','log.login').click()
time.sleep(1)

In [9]:
# 카페 접속
time.sleep(2)
moms_url = 'https://cafe.naver.com/firenze'
driver.get(moms_url)
time.sleep(1)

# 원하는 게시판 클릭
driver.find_element('xpath','//*[@id="menuLink123"]').click()
time.sleep(1)

# 프레임 전환
driver.switch_to.frame('cafe_main')
time.sleep(1)

### 게시물 크롤링
1. 첫번째 게시물 클릭 : XPath 활용
2. 크롤링할 개수 지정
3. 크롤링

In [10]:
import tqdm as tqdm

In [11]:
!pip install lxml

Defaulting to user installation because normal site-packages is not writeable


In [12]:
# 제목, 본문, 댓글, 날짜 list 생성
titles = []
contents = []
reviews = []
dates = []
count_views = 0

# 1번
driver.find_element('xpath','//*[@id="main-area"]/div[4]/table/tbody/tr[1]/td[1]/div[2]/div/a').click()

# 2번 : for문으로 지정한 게시물 개수까지 돌림
# 많은 양 크롤링할 경우 메모리 과부하로 크롤링이 멈출 수 있음
# 이 때 이미 받은 data들이 날아가지 않도록 for문을 이용해 지정
for i in range(1):  # 새로 창 열기
    for i in range(30): # 한 번에 몇개의 게시물을 크롤링할 것인가, 컴퓨터 성능에 따라 개수 달라짐(~5000개)
        driver.implicitly_wait(30)
        # 제목 수집
        time.sleep(random.uniform(0.5,1))   # 크롤링하다가 막히는 걸 방지하기 위해 최대한 사람처럼 보이게 만들어 줌 
        title = driver.find_element('xpath','//*[@id="app"]/div/div/div[2]/div[1]/div[1]/div/h3').text
        titles.append(title)

        # 본문 수집
        # 3가지 경우 발생 : 기본적인 본문, 사진으로만 구성된 본문, 수정을 통해 다른 선택자 생김
        try:
            content = driver.find_element(By.CSS_SELECTOR,'div.se-module.se-module-text')
            contents.append(content)
        except:
            try:
                content = driver.find_element(By.CSS_SELECTOR,'div.ContentRenderer')
                contents.append(content)
                # 다른 선택자가 생긴 경우
            except:
                content=[]  # 본문이 사진으로만 구성된 경우 빈칸으로 가져옴(사진 가져오게 할 수도 있음)
                contents.append(content)

        # 댓글 수집(댓글이 있는 경우, 없는 경우)
        soup = bs(driver.page_source, 'lxml') # 현재 페이지 HTML 정보 가지오 오기
        iscomment = soup.find_all('span', class_='text_comment')
        if (len(iscomment)==0):   # 댓글 없는 경우
            review = []
        else:   # 댓글 있는 경우
            WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'text_comment')))
            # 웹드라이버를 통해 브라우저에서 최대 15초 기다려주고, text_comment(댓글 선택)
            soup = bs(driver.page_source, 'lxml')
            iscomment = soup.find_all('span', class_='text_comment')
            review = []
            for i in iscomment:
                review.append([i.get_text()])   # iscomment 할당한 text_comment에서 text만 review에 append

        reviews.append(review)

        # 날짜 수집
        date = driver.find_element(By.XPATH,'//*[@id="app"]/div/div/div[2]/div[1]/div[2]/div/div[2]/span[1]').text
        dates.append(date)

        # count_views 크롤링 진행상황(50개마다 출력)
        count_views = count_views + 1
        if (count_views%50==0):
            print("{}개 게시물 크롤링 완료".format(count_views))
        else:
            pass

        # 다음글 클릭(다음글 버튼 클릭해서 연속적으로 크롤링)
        try:
            driver.find_element(By.CSS_SELECTOR,'#app > div > div > div.ArticleTopBtns > div.right_area > a.BaseButton.btn_next.BaseButton--skinGray.size_default').click()
            driver.implicitly_wait(20)

        except: # 다음글 버튼 사라지는 문제점 발생, 크롬 close 후 다시 open
            current_url = driver.current_url    # 다음글 버튼이 사라져서 크롤링이 멈춘 url 가져오기
            driver.close()

            # 크롬창 다시열기
            driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)

            # 재로그인
            login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
            driver.get(login_url)
            driver.implicitly_wait(10)

            driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
            driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
            time.sleep(1)

            driver.find_element('id','log.login').click()
            time.sleep(1)

            driver.get(current_url)
            driver.switch_to.frame('cafe_main')
            time.sleep(4)

    # 크롬이 다시 열릴 때 가지고 올 다음글 링크(herf 추출)
    link = driver.find_element(By.XPATH,'//*[@id="app"]/div/div/div[1]/div[2]/a[1]').get_attribute('href')

    # 현재 크롬창 닫기
    driver.close()

    # 다시 열기
    driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)

    # 재로그인
    login_url = 'https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com'
    driver.get(login_url)
    driver.implicitly_wait(10)

    driver.execute_script("document.getElementsByName('id')[0].value = \'" + my_id + "\'")
    driver.execute_script("document.getElementsByName('pw')[0].value = \'" + my_pw + "\'")
    time.sleep(1)

    driver.find_element('id','log.login').click()
    time.sleep(1)

    # 다음 링크부터 가져오기
    driver.get(link)
    driver.switch_to.frame('cafe_main')

# 크롬 창 닫기
driver.quit()


C:\Users\user\AppData\Local\Temp\ipykernel_16584\1627537063.py:98: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)


In [14]:
# 제목 리뷰 댓글이 잘 크롤링 되었는지 확인_게시물 개수를 의미함
print(len(titles))
print(len(contents))
print(len(reviews))
print(len(dates))

reviews = []
dates = []

30
30
0
0
